In [ ]:
"""
------------------------------------------------------------------------------
About
In this data analysis, the average response time of mission calls (fire/ems) 
is calculated for each admin area.
------------------------------------------------------------------------------
"""

'\n------------------------------------------------------------------------------\nAbout\nIn this data analysis, the proportion of mission calls (fire/ems) \nthat reach the 10-minute response time is calculated for each admin area.\n\nThe sum of mission_count_ems, mission_count_fire and mission_count_technical_rescue \nwas taken as the total sum of mission calls.\n------------------------------------------------------------------------------\n'

In [57]:
"""
------------------------------------------------------------------------------
Libraries
------------------------------------------------------------------------------
"""

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

In [58]:
"""
------------------------------------------------------------------------------
Import data
------------------------------------------------------------------------------
"""

df = pd.read_csv("../data/raw/wyx48.csv", dtype={"planning_room_id":str})

df_pgr = pd.read_csv("../data/raw/PGR.csv", dtype={"PGR_ID": str})

df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,...,response_time_fire_time_to_first_pump_std,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std
0,1100101,Stülerstraße,426,293,216,3,33,9,182,21,...,110.888486,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826
1,1100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,...,136.057777,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133
2,1100103,Lützowstraße,697,534,373,6,36,29,331,10,...,153.350058,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947
3,1100104,Körnerstraße,294,255,177,2,21,18,142,2,...,56.869303,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610
4,1100205,Wilhelmstraße,581,500,332,5,51,30,276,23,...,91.957311,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975


In [59]:

df_pgr.rename(columns={"PGR_ID": "pgr_id", "PGR_NAME":"pgr_name"}, inplace=True)

df_pgr.head()


,pgr_id,pgr_name,BEZ,STAND,GROESSE_M2
0,0930,Treptow-Köpenick 3,9,01.01.2021,4.068962e+07
1,0760,Marienfelde / Lichtenrade,7,01.01.2021,1.919230e+07
2,0920,Treptow-Köpenick 2,9,01.01.2021,1.920230e+07
3,0220,Kreuzberg Süd,2,01.01.2021,4.746238e+06
4,1260,Märkisches Viertel,12,01.01.2021,4.783336e+06


In [60]:
#add missing 0 in planning_room_id (LOR)
df["planning_room_id"] = df["planning_room_id"].str.zfill(8)
df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,...,response_time_fire_time_to_first_pump_std,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std
0,01100101,Stülerstraße,426,293,216,3,33,9,182,21,...,110.888486,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826
1,01100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,...,136.057777,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133
2,01100103,Lützowstraße,697,534,373,6,36,29,331,10,...,153.350058,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947
3,01100104,Körnerstraße,294,255,177,2,21,18,142,2,...,56.869303,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610
4,01100205,Wilhelmstraße,581,500,332,5,51,30,276,23,...,91.957311,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975


In [61]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)

df["pgr_id"] = df["planning_room_id"].str[:4]

df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,...,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std,pgr_id
0,01100101,Stülerstraße,426,293,216,3,33,9,182,21,...,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826,0110
1,01100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,...,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133,0110
2,01100103,Lützowstraße,697,534,373,6,36,29,331,10,...,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947,0110
3,01100104,Körnerstraße,294,255,177,2,21,18,142,2,...,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610,0110
4,01100205,Wilhelmstraße,581,500,332,5,51,30,276,23,...,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975,0110


In [62]:
#merge orgiginal data frame with admin area data frame
df_merged= pd.merge(df, df_pgr, on= "pgr_id", how="left")

In [63]:
"""
------------------------------------------------------------------------------
response_time_ems_critical_mean (in seconds)
response_time_ems_critical_std (in seconds)

response_time_fire_time_to_first_pump_mean (in seconds)
response_time_fire_time_to_first_pump_std (in seconds)

planning_room_name
------------------------------------------------------------------------------
"""
#subset
df_responseTime = df_merged[["pgr_id", "pgr_name", "response_time_ems_critical_mean", "response_time_ems_critical_std"]]
df_responseTime.head()

,pgr_id,pgr_name,response_time_ems_critical_mean,response_time_ems_critical_std
0,0110,Zentrum,593.758242,179.061417
1,0110,Zentrum,616.063927,209.523958
2,0110,Zentrum,589.604230,207.859285
3,0110,Zentrum,570.260563,196.087536
4,0110,Zentrum,611.612319,201.953546


In [64]:
#response time ems
df_responseTime["response_time_ems_critical_mean"] = (df_responseTime["response_time_ems_critical_mean"] /60)
df_responseTime["response_time_ems_critical_std"] = (df_responseTime["response_time_ems_critical_std"] /60)

df_responseTime.head()

/var/folders/_f/022l68353h9c2pr_3t55ns1m0000gn/T/ipykernel_25008/3993853783.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_responseTime["response_time_ems_critical_mean"] = (df_responseTime["response_time_ems_critical_mean"] /60)
/var/folders/_f/022l68353h9c2pr_3t55ns1m0000gn/T/ipykernel_25008/3993853783.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_responseTime["response_time_ems_critical_std"] = (df_responseTime["response_time_ems_critical_std"] /60)


,pgr_id,pgr_name,response_time_ems_critical_mean,response_time_ems_critical_std
0,0110,Zentrum,9.895971,2.984357
1,0110,Zentrum,10.267732,3.492066
2,0110,Zentrum,9.826737,3.464321
3,0110,Zentrum,9.504343,3.268126
4,0110,Zentrum,10.193539,3.365892


In [65]:
#group by admin area "Prognoseraum"
df_responseTime = df_responseTime.groupby(["pgr_id", "pgr_name"]).agg({"response_time_ems_critical_mean": "mean", 
                                                "response_time_ems_critical_std": "mean"
                                                }).reset_index()

#round
df_responseTime = df_responseTime.round({
    "response_time_ems_critical_mean": 2,
    "response_time_ems_critical_std": 2
})

df_responseTime.head()

,pgr_id,pgr_name,response_time_ems_critical_mean,response_time_ems_critical_std
0,0110,Zentrum,9.56,3.22
1,0120,Moabit,9.71,3.13
2,0130,Gesundbrunnen,10.09,3.27
3,0140,Wedding,9.65,3.40
4,0210,Kreuzberg Nord,9.32,3.09


In [66]:
#minutes to minute and seconds
def minutes_to_min_sec(minutes):
    min_part = int(minutes)  
    sec_part = round((minutes - min_part) * 60)
    return min_part, sec_part

#apply function and save values in separate columns
df_responseTime[["ems_critical_mean_min", "ems_critical_mean_sec"]] = pd.DataFrame(
    df_responseTime["response_time_ems_critical_mean"].apply(minutes_to_min_sec).tolist(), index= df_responseTime.index)


In [67]:
#drop columns
df_responseTime = df_responseTime.drop(columns=["response_time_ems_critical_mean", "response_time_ems_critical_std"])

#sort descending
df_responseTime.sort_values(by= "ems_critical_mean_min", ascending=True, inplace=True)
df_responseTime

,pgr_id,pgr_name,ems_critical_mean_min,ems_critical_mean_sec
6,0230,Kreuzberg Ost,8,59
0,0110,Zentrum,9,34
32,0740,Tempelhof,9,57
31,0730,Friedenau,9,49
30,0720,Schöneberg Süd,9,18
29,0710,Schöneberg Nord,9,46
25,0610,Steglitz,9,57
18,0430,Charlottenburg Zentrum,9,38
15,0370,Südlicher Prenzlauer Berg,9,57
20,0450,Wilmersdorf Zentrum,9,19


In [68]:
#value response_time_ems_critical_mean
responseTimeEms = df_responseTime[["pgr_id", "pgr_name", "ems_critical_mean_min", "ems_critical_mean_sec"]]

In [69]:
#csv
responseTimeEms.to_csv("../data/csv/responseTimeEMS.csv", index=False)